In [ ]:
# Import the necessary modules
from pyspark.sql.functions import *
from ultis import *
from datetime import datetime
from decouple import config

spark = create_spark()

In [ ]:
spark.sql('''
SELECT
    A.CASE_ID,
    B.USER_ID,
    A.COMMENT,
    A.CASE_RATING,
    C.CASE_CATEGORY_ID,
    FROM_UTC_TIMESTAMP(A.CASE_CREATED_AT, 'GMT+7') AS CREATED_AT,
    FROM_UTC_TIMESTAMP(NOW(), 'GMT+7') AS LAST_EXECUTED_AT
FROM delta.`s3a://bronze/USER_RATING` A
LEFT JOIN delta.`s3a://silver/USER_DIM` B ON A.USER_ID = B.USER_ID
LEFT JOIN delta.`s3a://silver/CASE_CATEGORY_DIM` C ON C.CASE_CATEGORY_ID = MD5(CONCAT(A.CASE_CATEGORY_1, A.CASE_CATEGORY_2))
''').write.format('delta').mode('overwrite').save('s3a://silver/USER_RATING_FACT')

In [ ]:
spark.stop()